In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
import time
import requests
import pandas as pd


In [2]:
def init_browser():
    # @NOTE: Replace the path with your actual path to the chromedriver
    executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
    return Browser("chrome", **executable_path, headless=False)



In [ ]:
def scrape_info():

    #run above function 'init_browser'
    browser = init_browser()

    # Scrape the NASA Mars News Site and collect the latest News
    url = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
    browser.visit(url)

    time.sleep(1)

    # Scrape page into Soup
    html = browser.html
    soup = bs(html, "html.parser")

    # Get the News Info Section
    news_info = soup.find('div', class_='list_text')
    #print(news_info.text)

    # collect the latest News Title
    news_sec1 = news_info.find('div', class_='content_title')
    news_title = news_sec1.a.text
    #print(news_title)

    # news article Paragraph Text
    news_p = news_info.find('div', class_="article_teaser_body").text
    #print(news_p)

    # Close the browser after scraping
    browser.quit()

In [ ]:
    ### LETS GET THE JPL IMAGE!!
    # Visit the url for JPL Featured Space Image here

    #run above function 'init_browser'
    browser = init_browser()

    url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
    browser.visit(url)

    time.sleep(5)

    # Scrape page into Soup
    html = browser.html
    soup = bs(html, "html.parser")


    # Use splinter to click on main 'full image' button
    image_button = browser.find_by_id('full_image')

    image_button.click()
    time.sleep(1)

    # Clicking 'more info' button. Making way to large image info
    browser.find_by_css(".buttons .button").click()
    time.sleep(1)

    #image url for the current Featured Mars Image
    browser.find_by_css(".lede").click()
    time.sleep(1)

    #assign the url string to a variable called featured_image_url
    featured_image_url = browser.url

    # Close the browser after scraping
    browser.quit()

In [ ]:
### LETS GET THE MARS FUN FACTS!!

#run above function 'init_browser'
browser = init_browser()

url = "https://space-facts.com/mars/"
browser.visit(url)

time.sleep(5)

# Scrape page into Soup
html = browser.html
soup = bs(html, "html.parser")

In [ ]:
tables = pd.read_html(url)

In [ ]:
# Close the browser after scraping
browser.quit()

In [ ]:
df = tables[0]
## remove index
df.set_index([0], inplace=True)
df

In [ ]:
df.columns.name = None
df

In [ ]:
## HTML tables from DataFrames
mars_facts = df.to_html()
mars_facts

In [3]:
## LETS GET THE MARS hemisphere images

browser = init_browser()

url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url)

time.sleep(1)

# Scrape page into Soup
html = browser.html
soup = bs(html, "html.parser")

In [4]:
## Find the image section of the page to iterate thru
#results = browser.find_by_css(".collapsible results .item")
#results

In [17]:
## GET titles
hem_titles = soup.find_all("div", class_="description")
#just_titles = hem_titles.find(["h3"])
titles = []
for hem_titles in hem_titles:
    m_title = hem_titles.find_all('h3')
    for t in m_title:
        titles.append(t)

In [18]:
titles

[<h3>Cerberus Hemisphere Enhanced</h3>,
 <h3>Schiaparelli Hemisphere Enhanced</h3>,
 <h3>Syrtis Major Hemisphere Enhanced</h3>,
 <h3>Valles Marineris Hemisphere Enhanced</h3>]

In [19]:
## SET empty dictionary for the html
img_dict = {}
for t in titles:
    img_dict["title"].append(t)

KeyError: 'title'

In [15]:
print(img_dict)

{'title': <h3>Valles Marineris Hemisphere Enhanced</h3>}


In [ ]:
## GET 1st img_url
browser.find_by_css(".full-content .thumb")[0].click()
time.sleep(1)
browser.links.find_by_text("Sample").click()
time.sleep(1)
img_url = browser.url
img_dict["img_url"[0]] = img_url

# return home
browser.quit()

In [ ]:
browser = init_browser()

url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url)

time.sleep(1)

# Scrape page into Soup
html = browser.html
soup = bs(html, "html.parser")

In [ ]:
## GET 2nd img_url
browser.find_by_css(".full-content .thumb")[1].click()
time.sleep(1)
browser.links.find_by_text("Sample").click()
time.sleep(1)
img_url = browser.url
img_dict["img_url"[1]] = img_url

# return home
browser.quit()

In [ ]:
browser = init_browser()

url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url)

time.sleep(1)

# Scrape page into Soup
html = browser.html
soup = bs(html, "html.parser")

In [ ]:
## GET 3rd img_url
browser.find_by_css(".full-content .thumb")[2].click()
time.sleep(1)
browser.links.find_by_text("Sample").click()
time.sleep(1)
img_url = browser.url
img_dict["img_url"[2]] = img_url

# return home
browser.quit()

In [ ]:
browser = init_browser()

url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url)

time.sleep(1)

# Scrape page into Soup
html = browser.html
soup = bs(html, "html.parser")

In [ ]:
## GET 4rth img_url
browser.find_by_css(".full-content .thumb")[3].click()
time.sleep(1)
browser.links.find_by_text("Sample").click()
time.sleep(1)
img_url = browser.url
img_dict["img_url"[3]] = img_url

# return home
browser.quit()

In [ ]:
print(img_dict)

In [ ]:
# Close the browser after scraping
browser.quit()

In [ ]:
    
    
    
    
    
    
    # Store data in a dictionary
    image_dict = image_dict
    
    mars_data = {
        "news_title": news_title,
        "news_p": news_p,
        "featured_image_url": featured_image_url,
        "mars_facts": mars_facts,
        "img_list": img_list
    }


    return mars_data